# Домашняя работа к лекции «Продвинутый pandas»

## Задание 1

Для датафрейма log из материалов занятия создайте столбец source_type по следующим правилам:

если источник traffic_source равен yandex или google, то в source_type ставится organic
для источников paid и email из России - ставим ad
для источников paid и email не из России - ставим other
все остальные варианты берем из traffic_source без изменений

In [1]:
import pandas as pd
import re

In [2]:
log = pd.read_csv('visit_log.csv', sep=';')

def source_type(row):
    row_ts = row['traffic_source']
    row_reg = row['region']
    for source in row:
        if ('yandex' == row_ts) or ('google' == row_ts):
            return 'organic'
        elif ('paid' == row_ts) and (row_reg == 'Russia') or ('email' == row_ts) and (row_reg == 'Russia'):
            return 'ad'
        elif ('paid' == row_ts) and (row_reg != 'Russia') or ('email' == row_ts) and (row_reg != 'Russia'):
            return 'other'
        else:
            return row_ts
        
log['source_type'] = log.apply(source_type, axis=1)
log.head(10)

,timestamp,visit_id,url,region,user_id,traffic_source,source_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic
5,1549980742,8855508aad,https://host.ru/df646c3676cc259fa0,Russia,fc43898e47,yandex,organic
6,1549980742,b0f66adc83,https://host.ru/b8b58337d272ee7b15,Russia,13fc55e781,paid,ad
7,1549980754,837885c8f8,https://host.ru/108ce4b365afb7b88e,Russia,cb5082b6f6,direct,direct
8,1549980760,af5570f5a1,https://host.ru/3004a8273caeef2867,China,45664f7af2,direct,direct
9,1549980765,3e7077fd2f,https://host.ru/df646c3676cc259fa0,Russia,6f9de8c8b6,email,ad


## Задание 2

В файле URLs.txt содержатся url страниц новостного сайта. Вам необходимо отфильтровать его по адресам страниц с текстами новостей. Известно, что шаблон страницы новостей имеет внутри url следующую конструкцию: /, затем 8 цифр, затем дефис. Выполните следующие действия:
- Прочитайте содержимое файла с датафрейм;
- Отфильтруйте страницы с текстом новостей, используя метод str.contains и регулярное выражение в соответствии с заданным шаблоном

In [3]:
# В этом решении через длинну строк отсортированы url, заполнен шаблоном столбец 'news_url' и по нему отфильтрованы строки по адресам с текстами новостей.

In [4]:
urls = pd.read_csv('URLs.txt')
urls['len_str'] = urls['url'].apply(lambda string: len(string))
urls = urls.sort_values('len_str', ascending=True)
for string in urls.url:
    tamplate = str(re.findall(r'\/\d{8}\-', string))[2:-2]
    urls.loc[urls.url.str.contains(tamplate), 'news_url'] = tamplate
    urlsf = urls.loc[urls.news_url != '']
urlsf[['url']].head(20)

,url
45,/video/36151719-petrosyanu-grozit-sud/
59,/articles/36213214-vozmozhna-bolshaya-voyna/
16,/starlife/36174817-guzeeva-rasstalas-s-muzhem/
57,/starlife/36240935-dzhigurde-postavili-diagnoz/
47,/articles/36193368-donbass-reshil-udarit-pervym/
41,/cis/36189830-kiev-arestoval-imuschestvo-v-krymu/
54,/finances/36043911-rublyu-predrekli-dvoynoy-udar/
60,/articles/36142564-ukrainu-ureguliruyut-po-tih...
66,/europe/36046038-v-parizhskom-metro-progremel-...
19,/cis/36229699-poroshenko-obyavil-vseukrainskiy...


## Задание 3

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [5]:
ratings = pd.read_csv('ratings.csv')

ratings_pivot_table = ratings.groupby('userId').agg({'timestamp': ['min', 'max', 'count']})['timestamp'].\
                      sort_values('count', ascending=False)
lifetime_pivot_table = ratings_pivot_table.loc[ratings_pivot_table['count'] > 100, 'lifetime'] = \
                      (ratings_pivot_table['max'] - ratings_pivot_table['min']).mean()
lifetime_pivot_table

17373661.00447094

## Задание 4

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
- таблицу с тремя типами выручки для каждого client_id без указания адреса клиента;
- аналогичную таблицу по типам выручки с указанием адреса клиента

##### Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

Дана статистика услуг перевозок клиентов компании по типам:

- rzd - железнодорожные перевозки
- auto - автомобильные перевозки
- air - воздушные перевозки
- client_base - адреса клиентов

In [6]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [7]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [8]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [9]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [10]:
all_revenue_base = client_base[['client_id']].merge(rzd, how='left', on='client_id').\
                                              merge(auto, how='left', on='client_id').\
                                              merge(air, how='left', on='client_id')
all_revenue_base

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [11]:
all_revenue_base_add = client_base.merge(rzd, how='left', on='client_id').\
                                   merge(auto, how='left', on='client_id').\
                                   merge(air, how='left', on='client_id')
all_revenue_base_add

,client_id,address,rzd_revenue,auto_revenue,air_revenue
0,111,Комсомольская 4,1093.0,NaN,NaN
1,112,Энтузиастов 8а,2810.0,NaN,NaN
2,113,Левобережная 1а,10283.0,57483.0,NaN
3,114,Мира 14,5774.0,83.0,NaN
4,115,ЗЖБИиДК 1,981.0,912.0,81.0
5,116,Строителей 18,NaN,4834.0,4.0
6,117,Панфиловская 33,NaN,98.0,13.0
7,118,Мастеркова 4,NaN,NaN,173.0
